In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
from sklearn.preprocessing import MinMaxScaler

# ---------------------------
# Load the trained model
# ---------------------------
model = load_model('backend/models/random_forest_stock_model.h5')

# ---------------------------
# Load and prepare your dataset
# ---------------------------
# Update the path to your dataset as needed
df = pd.read_csv('../data/updated_data.csv')
df['Date'] = pd.to_datetime(df['Date'])

# Choose the symbol you want to predict (e.g., AAPL)
symbol = 'AAPL'
symbol_df = df[df['Symbol'] == symbol].sort_values('Date')

# Define features as used during training
features = ['Open', 'High', 'Low', 'Close', 'Volume']

# Recreate the scaler – ideally you would have saved it along with your model.
scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(df[features].values)

# ---------------------------
# Create a sequence for prediction
# ---------------------------
sequence_length = 60
if len(symbol_df) < sequence_length:
    raise ValueError("Not enough data to form a sequence for prediction.")

# Use the last 'sequence_length' days for prediction
latest_data = symbol_df[features].values[-sequence_length:]
latest_data_scaled = scaler.transform(latest_data).reshape(1, sequence_length, len(features))

# ---------------------------
# Make a prediction
# ---------------------------
predicted_scaled = model.predict(latest_data_scaled)
predicted_scaled_value = predicted_scaled[0, 0]  # predicted "Close" price (scaled)

# To inverse transform, we create a dummy array with the correct shape and set the 'Close' column
dummy = np.zeros((1, len(features)))
dummy[0, 3] = predicted_scaled_value  # index 3 corresponds to 'Close'
predicted_price = scaler.inverse_transform(dummy)[0, 3]

# Current price for comparison
current_price = symbol_df['Close'].iloc[-1]
price_change_percentage = ((predicted_price - current_price) / current_price) * 100

print(f"Current price for {symbol}: ${current_price:.2f}")
print(f"Predicted next price for {symbol}: ${predicted_price:.2f}")
print(f"Predicted price change: {price_change_percentage:.2f}%")

# Determine trading decision
threshold = 0.01  # 1% change threshold
if price_change_percentage > threshold:
    decision = "BUY"
elif price_change_percentage < -threshold:
    decision = "SELL"
else:
    decision = "HOLD"
print(f"Trading Decision: {decision}")

# ---------------------------
# Plot the results with improved aesthetics
# ---------------------------
# Choose how many past days to display (if available)
n_days = 100
plot_data = symbol_df[-n_days:].copy()

# Prepare dates and prices for plotting
dates = plot_data['Date'].tolist()
actual_prices = plot_data['Close'].values

# Append next day (for prediction)
next_date = dates[-1] + pd.Timedelta(days=1)
dates.append(next_date)

plt.figure(figsize=(14, 7))

# Plot historical prices
plt.plot(dates[:-1], actual_prices, marker='o', linestyle='-', label='Historical Price', color='blue')

# Plot the predicted price as a dashed line from the last actual day to the predicted next day
plt.plot([dates[-2], dates[-1]], [actual_prices[-1], predicted_price],
         marker='o', linestyle='--', label='Predicted Price', color='red')

plt.xlabel('Date', fontsize=12)
plt.ylabel('Price', fontsize=12)
plt.title(f'{symbol} Historical Prices and Next Price Prediction (RNN Model)', fontsize=14)
plt.xticks(rotation=45)
plt.legend(fontsize=12)
plt.tight_layout()
plt.show()

ValueError: No model config found in the file at <tensorflow.python.platform.gfile.GFile object at 0x321386810>.